In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import geopandas as gpd
from census import Census
from us import states

import config

In [2]:
# Set API key
c = Census(config.key)

In [3]:
# Obtain Census variables from the 2014 ACS at the tract level for the Commonwealth of Illinois (FIPS code: 17)
# Sources: https://api.census.gov/data/2014/acs/acs5/variables.html; https://pypi.org/project/census/
il_census = c.acs5.state_county_tract(fields = ('NAME', 'B23025_001E', 'B23025_002E', 'B23025_007E', 'C17002_001E', 'C17002_002E', 'C17002_003E', 'B15003_001E', 'B15003_002E', 'B15003_003E', 'B15003_004E', 'B15003_005E', 'B15003_006E', 'B15003_007E', 'B15003_008E', 'B15003_009E', 'B15003_010E', 'B15003_011E', 'B15003_012E', 'B15003_013E', 'B15003_014E', 'B15003_015E', 'B15003_016E', 'B15003_017E', 'B15003_018E', 'B15003_019E', 'B15003_020E', 'B15003_021E', 'B15003_022E', 'B15003_023E', 'B15003_024E', 'B15003_025E','B01003_001E'),
                                      state_fips = states.IL.fips,
                                      county_fips = "031", # Cook County
                                      tract = "*",
                                      year = 2014)

In [4]:
# Create a dataframe from the census data
il_df = pd.DataFrame(il_census)

# Show the dataframe
print(il_df.head(2))
print('Shape: ', il_df.shape)

                                          NAME  B23025_001E  B23025_002E  \
0     Census Tract 2407, Cook County, Illinois       1158.0        837.0   
1  Census Tract 8221.02, Cook County, Illinois       4050.0       2440.0   

   B23025_007E  C17002_001E  C17002_002E  C17002_003E  B15003_001E  \
0        321.0       1427.0         66.0        440.0       1047.0   
1       1610.0       4580.0        182.0        249.0       3639.0   

   B15003_002E  B15003_003E  ...  B15003_020E  B15003_021E  B15003_022E  \
0         36.0          0.0  ...        119.0         36.0        340.0   
1         55.0          0.0  ...        534.0        185.0        356.0   

   B15003_023E  B15003_024E  B15003_025E  B01003_001E  state  county   tract  
0        140.0         20.0         32.0       1427.0     17     031  240700  
1        296.0         51.0         44.0       4752.0     17     031  822102  

[2 rows x 36 columns]
Shape:  (1319, 36)


In [5]:
# Access shapefile of Illinois census tracts
il_tract = gpd.read_file("https://www2.census.gov/geo/tiger/TIGER2014/TRACT/tl_2014_17_tract.zip")

# Reproject shapefile to UTM Zone 16N
# https://spatialreference.org/ref/epsg/wgs-84-utm-zone-16n/
il_tract = il_tract.to_crs(epsg = 32616)

# Print GeoDataFrame of shapefile
print(il_tract.head(2))
print('Shape: ', il_tract.shape)

# Check shapefile projection
print("\nThe shapefile projection is: {}".format(il_tract.crs))

  STATEFP COUNTYFP TRACTCE        GEOID NAME          NAMELSAD  MTFCC  \
0      17      161  022800  17161022800  228  Census Tract 228  G5020   
1      17      161  022900  17161022900  229  Census Tract 229  G5020   

  FUNCSTAT    ALAND  AWATER     INTPTLAT      INTPTLON  \
0        S  2097335       0  +41.4990973  -090.5473216   
1        S  1311567       0  +41.4897868  -090.5477824   

                                            geometry  
0  POLYGON ((203053.165 4599746.440, 203054.056 4...  
1  POLYGON ((203043.102 4599640.181, 203046.622 4...  
Shape:  (3123, 13)

The shapefile projection is: epsg:32616


In [6]:
# Combine state, county, and tract columns together to create a new string and assign to new column
il_df["GEOID"] = il_df["state"] + il_df["county"] + il_df["tract"]

In [7]:
# Print head of dataframe
il_df.head()

,NAME,B23025_001E,B23025_002E,B23025_007E,C17002_001E,C17002_002E,C17002_003E,B15003_001E,B15003_002E,B15003_003E,...,B15003_021E,B15003_022E,B15003_023E,B15003_024E,B15003_025E,B01003_001E,state,county,tract,GEOID
0,"Census Tract 2407, Cook County, Illinois",1158.0,837.0,321.0,1427.0,66.0,440.0,1047.0,36.0,0.0,...,36.0,340.0,140.0,20.0,32.0,1427.0,17,031,240700,17031240700
1,"Census Tract 8221.02, Cook County, Illinois",4050.0,2440.0,1610.0,4580.0,182.0,249.0,3639.0,55.0,0.0,...,185.0,356.0,296.0,51.0,44.0,4752.0,17,031,822102,17031822102
2,"Census Tract 2226, Cook County, Illinois",1308.0,988.0,320.0,1554.0,152.0,106.0,1050.0,17.0,0.0,...,63.0,250.0,78.0,8.0,14.0,1554.0,17,031,222600,17031222600
3,"Census Tract 2308, Cook County, Illinois",992.0,618.0,374.0,1194.0,31.0,148.0,822.0,35.0,0.0,...,64.0,165.0,14.0,7.0,0.0,1195.0,17,031,230800,17031230800
4,"Census Tract 202, Cook County, Illinois",4649.0,2923.0,1726.0,6255.0,487.0,490.0,4166.0,125.0,0.0,...,160.0,1414.0,540.0,100.0,39.0,6255.0,17,031,020200,17031020200


In [8]:
# Remove columns
il_df = il_df.drop(columns = ["state", "county", "tract"])

# Show updated dataframe
il_df.head(2)

,NAME,B23025_001E,B23025_002E,B23025_007E,C17002_001E,C17002_002E,C17002_003E,B15003_001E,B15003_002E,B15003_003E,...,B15003_018E,B15003_019E,B15003_020E,B15003_021E,B15003_022E,B15003_023E,B15003_024E,B15003_025E,B01003_001E,GEOID
0,"Census Tract 2407, Cook County, Illinois",1158.0,837.0,321.0,1427.0,66.0,440.0,1047.0,36.0,0.0,...,0.0,24.0,119.0,36.0,340.0,140.0,20.0,32.0,1427.0,17031240700
1,"Census Tract 8221.02, Cook County, Illinois",4050.0,2440.0,1610.0,4580.0,182.0,249.0,3639.0,55.0,0.0,...,198.0,200.0,534.0,185.0,356.0,296.0,51.0,44.0,4752.0,17031822102


In [9]:
# Check column data types for census data
print("Column data types for census data:\n{}".format(il_df.dtypes))

# Check column data types for census shapefile
print("\nColumn data types for census shapefile:\n{}".format(il_tract.dtypes))

# Source: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.dtypes.html

Column data types for census data:
NAME            object
B23025_001E    float64
B23025_002E    float64
B23025_007E    float64
C17002_001E    float64
C17002_002E    float64
C17002_003E    float64
B15003_001E    float64
B15003_002E    float64
B15003_003E    float64
B15003_004E    float64
B15003_005E    float64
B15003_006E    float64
B15003_007E    float64
B15003_008E    float64
B15003_009E    float64
B15003_010E    float64
B15003_011E    float64
B15003_012E    float64
B15003_013E    float64
B15003_014E    float64
B15003_015E    float64
B15003_016E    float64
B15003_017E    float64
B15003_018E    float64
B15003_019E    float64
B15003_020E    float64
B15003_021E    float64
B15003_022E    float64
B15003_023E    float64
B15003_024E    float64
B15003_025E    float64
B01003_001E    float64
GEOID           object
dtype: object

Column data types for census shapefile:
STATEFP       object
COUNTYFP      object
TRACTCE       object
GEOID         object
NAME          object
NAMELSAD      object
MT

In [10]:
# Join the attributes of the dataframes together
# Source: https://geopandas.org/docs/user_guide/mergingdata.html
il_merge = il_tract.merge(il_df, on = "GEOID")

# Show result
print(il_merge.head(2))
print('Shape: ', il_merge.shape)

  STATEFP COUNTYFP TRACTCE        GEOID  NAME_x             NAMELSAD  MTFCC  \
0      17      031  030101  17031030101  301.01  Census Tract 301.01  G5020   
1      17      031  030701  17031030701  307.01  Census Tract 307.01  G5020   

  FUNCSTAT   ALAND  AWATER  ... B15003_017E B15003_018E B15003_019E  \
0        S  122359  115695  ...       267.0        46.0        36.0   
1        S   69878       0  ...       157.0        15.0        82.0   

  B15003_020E  B15003_021E  B15003_022E  B15003_023E  B15003_024E  \
0       586.0        149.0       1081.0        551.0         67.0   
1       185.0        144.0        408.0        196.0         65.0   

   B15003_025E  B01003_001E  
0         72.0       3727.0  
1         13.0       1734.0  

[2 rows x 46 columns]
Shape:  (1319, 46)


In [11]:
# Create new dataframe from select columns
il_all_tract = il_merge[["STATEFP", "COUNTYFP", "TRACTCE", "GEOID", "geometry", 'B15003_001E', 'B15003_002E', 'B15003_003E', 'B15003_004E', 'B15003_005E', 'B15003_006E', 'B15003_007E', 'B15003_008E', 'B15003_009E', 'B15003_010E', 'B15003_011E', 'B15003_012E', 'B15003_013E', 'B15003_014E', 'B15003_015E', 'B15003_016E', 'B15003_017E', 'B15003_018E', 'B15003_019E', 'B15003_020E', 'B15003_021E', 'B15003_022E', 'B15003_023E', 'B15003_024E', 'B15003_025E', "C17002_001E", "C17002_002E", "C17002_003E", 'B23025_001E', 'B23025_002E', 'B23025_007E', "B01003_001E"]]

# Show dataframe
print(il_all_tract.head(2))
print('Shape: ', il_all_tract.shape)

  STATEFP COUNTYFP TRACTCE        GEOID  \
0      17      031  030101  17031030101   
1      17      031  030701  17031030701   

                                            geometry  B15003_001E  \
0  POLYGON ((445305.561 4649130.618, 445324.697 4...       3190.0   
1  POLYGON ((445320.511 4648168.487, 445339.650 4...       1362.0   

   B15003_002E  B15003_003E  B15003_004E  B15003_005E  ...  B15003_023E  \
0         58.0          0.0          0.0          0.0  ...        551.0   
1          4.0          0.0          0.0          0.0  ...        196.0   

   B15003_024E  B15003_025E  C17002_001E  C17002_002E  C17002_003E  \
0         67.0         72.0       3727.0        323.0        577.0   
1         65.0         13.0       1555.0        174.0        194.0   

   B23025_001E  B23025_002E  B23025_007E  B01003_001E  
0       3467.0       2264.0       1203.0       3727.0  
1       1542.0       1108.0        434.0       1734.0  

[2 rows x 37 columns]
Shape:  (1319, 37)


In [12]:
# merge columns together
il_all_tract["less_or_8th_grade"] = (il_all_tract['B15003_002E'] + il_all_tract['B15003_003E'] + il_all_tract['B15003_004E'] + il_all_tract[ 'B15003_005E'] + il_all_tract['B15003_006E'] + il_all_tract[ 'B15003_007E'] + il_all_tract['B15003_008E'] + il_all_tract[ 'B15003_009E'] + il_all_tract['B15003_010E'] + il_all_tract[ 'B15003_011E'] + il_all_tract['B15003_012E'])
il_all_tract["less_or_12th_grade"] = (il_all_tract[ 'B15003_013E']+ il_all_tract['B15003_014E'] + il_all_tract[ 'B15003_015E'] + il_all_tract['B15003_016E'])
il_all_tract["highschool or equivalent"] = (il_all_tract[ 'B15003_017E'] + il_all_tract['B15003_018E'])
il_all_tract["some college"] = (il_all_tract[ 'B15003_019E'] + il_all_tract['B15003_020E'])
il_all_tract["associate"] = (il_all_tract[ 'B15003_021E'])
il_all_tract["bachelor"] = (il_all_tract['B15003_022E'])
il_all_tract["master or higher"] = (il_all_tract[ 'B15003_023E'] + il_all_tract['B15003_024E'] + il_all_tract[ 'B15003_025E'])

/home/avasd/.local/lib/python3.8/site-packages/geopandas/geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/home/avasd/.local/lib/python3.8/site-packages/geopandas/geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/home/avasd/.local/lib/python3.8/site-packages/geopandas/geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,

In [13]:
# Get education rate and store values in new column
il_all_tract["Education_Rate"] = (il_all_tract["less_or_8th_grade"] + il_all_tract["less_or_12th_grade"] + il_all_tract["highschool or equivalent"] + il_all_tract["some college"] + il_all_tract["associate"] + il_all_tract["bachelor"] + il_all_tract["master or higher"]) / il_all_tract["B01003_001E"] * 100
il_all_tract["Employment_Rate"] = (il_all_tract["B23025_002E"] + il_all_tract["B23025_007E"]) / il_all_tract["B01003_001E"] * 100
il_all_tract["Poverty_Rate"] = (il_all_tract["C17002_002E"] + il_all_tract["C17002_003E"]) / il_all_tract["B01003_001E"] * 100

# Show dataframe
il_all_tract.columns

/home/avasd/.local/lib/python3.8/site-packages/geopandas/geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


Index(['STATEFP', 'COUNTYFP', 'TRACTCE', 'GEOID', 'geometry', 'B15003_001E',
       'B15003_002E', 'B15003_003E', 'B15003_004E', 'B15003_005E',
       'B15003_006E', 'B15003_007E', 'B15003_008E', 'B15003_009E',
       'B15003_010E', 'B15003_011E', 'B15003_012E', 'B15003_013E',
       'B15003_014E', 'B15003_015E', 'B15003_016E', 'B15003_017E',
       'B15003_018E', 'B15003_019E', 'B15003_020E', 'B15003_021E',
       'B15003_022E', 'B15003_023E', 'B15003_024E', 'B15003_025E',
       'C17002_001E', 'C17002_002E', 'C17002_003E', 'B23025_001E',
       'B23025_002E', 'B23025_007E', 'B01003_001E', 'less_or_8th_grade',
       'less_or_12th_grade', 'highschool or equivalent', 'some college',
       'associate', 'bachelor', 'master or higher', 'Education_Rate',
       'Employment_Rate', 'Poverty_Rate'],
      dtype='object')

In [15]:
df = pd.read_csv('../data/preprocessed_tract.csv', low_memory=False)
il_all_tract['GEOID'] = il_all_tract['GEOID'].astype(np.float64)

In [16]:
df = pd.merge(df, il_all_tract, how='inner', left_on = 'CENSUS TRACT', right_on = 'GEOID')
df = df.drop(['STATEFP', 'COUNTYFP', 'TRACTCE', 'GEOID', 'geometry', 'C17002_001E', 'C17002_002E', 'C17002_003E', 'B15003_001E', 'B15003_002E', 'B15003_003E', 'B15003_004E', 'B15003_005E', 'B15003_006E', 'B15003_007E', 'B15003_008E', 'B15003_009E', 'B15003_010E', 'B15003_011E', 'B15003_012E', 'B15003_013E', 'B15003_014E', 'B15003_015E', 'B15003_016E', 'B15003_017E', 'B15003_018E', 'B15003_019E', 'B15003_020E', 'B15003_021E', 'B15003_022E', 'B15003_023E', 'B15003_024E', 'B15003_025E', 'B01003_001E', 'B23025_001E', 'B23025_002E', 'B23025_007E', 'less_or_8th_grade', 'less_or_12th_grade', 'highschool or equivalent', 'some college', 'associate', 'bachelor', 'master or higher', 'CENSUS TRACT'], axis=1)
df = df.dropna()
df.shape

(223839, 18)

In [17]:
df.to_csv('../data/all_census_data.csv', index=False)